<a href="https://colab.research.google.com/github/LuisManuelCatzoliSoriano/EDP-I/blob/main/M%C3%A9tododeC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

In [6]:
x = sp.symbols('x') # variable
u = sp.Function('u') # función

Para resolver la ecuación (poner la ecuación)

In [8]:
ode = u(x).diff(x,x) + u(x) + x
sp.dsolve(ode,u(x))

Eq(u(x), C1*sin(x) + C2*cos(x) - x)

In [9]:
sol = sp.dsolve(ode, ics={u(0):0,u(1):0})

In [10]:
sol

Eq(u(x), -x + sin(x)/sin(1))